<a href="https://colab.research.google.com/github/jsgf04/TdR_Physics_behind_skiing/blob/main/P%C3%A8ndols%20amb%20el%20pivot%20m%C3%B2bil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numpy import sin,cos,sign,sqrt,arcsin,pi, arctan, tan
from scipy.integrate import solve_ivp
from matplotlib import pyplot as plt
from termcolor import colored
from matplotlib.pyplot import figure
import sympy as smp
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib import animation, rc
from IPython.display import HTML
 
print(colored("ADVERTÈNCIA: EL SIMULADOR ÉS VÀLID FINS APROXIMADAMENT 65º-70º","red"))
g = 9.807
height = float(input("Alçada de l'esquiador (m): "))
l = 0.56*height
R_sc = float(input("Radi dels esquís (m): "))
V = float(input("Velocitat de l'esquiador durant el trajecte (m/s): "))
 
if V**2/(g*R_sc)<1:
  Psi_eq = round(np.arcsin(V**2/(g*R_sc))*(180/np.pi), 3)
else:
  Psi_eq = "No hi han posicions d'equilibri"
 
if V**2/(g*R_sc)<1:
  text2 = "${\Psi}_{equilibri}$ = ±"
else:
  text2 = "${\Psi}_{equilibri}$ = "
 
def equations(t,y):
    dy=np.zeros([2])
    dy[0] = y[1]
    dy[1] = (g/l)*np.sin(y[0])-((V**2)/(l*R_sc))*np.sign(y[0])
 
    return dy
 
final_time = int(input("Temps final o durada del trajecte (s): "))
interval=30*final_time
time = np.linspace(0, final_time, interval)
 
if V**2/(g*R_sc)<1:
  Psi_equilibri_arr = round(0.98*Psi_eq,3)
  print("Angle d'equilibri per referència: ±"+str(Psi_eq)+"º",  colored(("[NO POT PASSAR DE ± 0.99 · \u03A8  ≈ ± "+str(Psi_equilibri_arr)+"... º]"),"red"))
else:
  print("No hi han posicions d'equilibri, és a dir, el pèndol/l'esquiador oscil·larà constantment a qualsevol angle sempre i quan no sigui extrem (70º-90º, 90º seria inclinar-se paral·lelament amb el terra | A partir de 70º l'esquiador pateix fins a 3 g s de força en cas el cas que la pista sigui horitzontal, això requereix molta força muscular.)")
 
initial_Psi = float(input("Angle inicial del pèndol/esquiador respecte de la vertical, \u03A6 (º): "))
y0 = [np.radians(initial_Psi), np.radians(0.0)]
 
res = solve_ivp(equations, (0,final_time), y0, method='RK45', t_eval=time, dense_output=True, rtol=1e-8, atol=1e-10)
 
t, psi, omega = res.t, res.y[0], res.y[1]
 
dense1 = res.sol
def dgamma(t, gamma):
    Psi, Omega = dense1(t)

    return [(V*np.sign(Psi))/(R_sc*np.cos(Psi))]

initial_gamma = [np.radians(0.0)]

gamma_res = solve_ivp(dgamma, (0,final_time), initial_gamma, method="RK45", t_eval = time, dense_output=True, rtol=1e-8, atol=1e-10)
gamma_times, gammas = gamma_res.t, gamma_res.y

dense2 = gamma_res.sol
def dy(t, y):
    Gamma = dense2(t)

    return [V*np.sin(Gamma)]

initial_y = [0.0]

y_res = solve_ivp(dy, (0,final_time), initial_y, method="RK45", t_eval = time, dense_output=True, rtol=1e-8, atol=1e-10)
y_times, y_skis = y_res.t, y_res.y[0]


pos_y= l*np.sin(psi)+y_skis
pos_z= l*np.cos(psi)
 
fig = plt.figure()
ax = plt.axes()
gca = plt.gca()
gca.set_aspect("equal")
plt.vlines(x= (max(y_skis))/2,ymin=0, ymax=2,colors="firebrick",linestyles="--",linewidth=1, zorder=0)
plt.hlines(y=0,xmin=-1, xmax=max(y_skis)+1,colors="black",linewidth=1,zorder=0)
plt.scatter(100,100,26,c="red", label="CM",zorder=2)
plt.xlabel("X [m]", fontsize=9, fontweight = "bold")
plt.ylabel("Z [m]", fontsize=9, fontweight = "bold")
plt.title('Moviment del pèndol centrífug invertit ')
gca.axes.xaxis.set_ticks([])
gca.axes.yaxis.set_ticks([l])
plt.ylim(-0.25,l+0.4)
plt.xlim(-0.25,max(y_skis)+0.25)
plt.legend(loc = "upper right", fontsize=8)
trans = plt.axes().transAxes
time_text1 = ax.text(0.01, 0.93, '', transform=trans, fontsize=7)
psi_text = ax.text(0.02, 0.85, '', transform=trans, fontsize=7)
line, = plt.plot([], [], 'k', lw=0.4, zorder=1)
skis_frontal, = plt.plot([], [], 'k', lw=2, zorder=1)
point, = plt.plot([],[], marker="o", c="red")
plt.close()

skis_Y_ini = -0.07*np.cos(psi)+y_skis
skis_Y_fin = 0.07*np.cos(psi)+y_skis
skis_Z_ini = 0.07*np.sin(psi)
skis_Z_fin = -0.07*np.sin(psi)

def init():
  line.set_data([], [])
  skis_frontal.set_data([], [])
  point.set_data([],[])
  time_text1.set_text('')
  psi_text.set_text('')
  return line, skis_frontal, point, time_text1, psi_text

def animate(i):
  line.set_data([y_skis[i],pos_y[i]], [0,pos_z[i]])
  skis_frontal.set_data([skis_Y_ini[i], skis_Y_fin[i]], [skis_Z_ini[i], skis_Z_fin[i]])
  point.set_data([pos_y[i]],[pos_z[i]])
  time_text1.set_text('Temps = %.1f s' %(i/30))
  psi_text.set_text('\u03A6 = \u03A8 = %.2f º' %np.degrees(psi[i]))
  return line, skis_frontal, point, time_text1, psi_text
 
anim = FuncAnimation(fig, animate,init_func=init, frames=interval, interval=33.3333334, blit=True)
 
if V**2/(g*R_sc)<1 and (initial_Psi > Psi_eq or initial_Psi < -Psi_eq):
  adv1 = "(L'esquiador ha perdut l'equilibri i ha caigut)"
else:
  adv1 = " "
 
print(colored(adv1,"red",attrs=['bold']))
HTML(anim.to_html5_video())

ADVERTÈNCIA: EL SIMULADOR ÉS VÀLID FINS APROXIMADAMENT 65º-70º
Alçada de l'esquiador (m): 1.8
Radi dels esquís (m): 14
Velocitat de l'esquiador durant el trajecte (m/s): 12
Temps final o durada del trajecte (s): 10
No hi han posicions d'equilibri, és a dir, el pèndol/l'esquiador oscil·larà constantment a qualsevol angle sempre i quan no sigui extrem (70º-90º, 90º seria inclinar-se paral·lelament amb el terra | A partir de 70º l'esquiador pateix fins a 3 g s de força en cas el cas que la pista sigui horitzontal, això requereix molta força muscular.)
Angle inicial del pèndol/esquiador respecte de la vertical, Φ (º): 45


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:94: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


In [ ]:
import numpy as np
from numpy import sin,cos,sign,sqrt,arcsin,pi, arctan, tan
from scipy.integrate import solve_ivp
from matplotlib import pyplot as plt
from termcolor import colored
from matplotlib.pyplot import figure
import sympy as smp
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib import animation, rc
from IPython.display import HTML

print(colored("ADVERTÈNCIA: EL SIMULADOR ÉS VÀLID FINS APROXIMADAMENT 65º-70º","red"))
g = 9.807
height = float(input("Alçada de l'esquiador (m): "))
l_o = 0.56*height
R_sc = float(input("Radi dels esquís (m): "))
V = float(input("Velocitat de l'esquiador durant el trajecte (m/s): "))
 
if V**2/(g*R_sc)<1:
  Psi_eq = round(np.arcsin(V**2/(g*R_sc))*(180/np.pi), 3)
else:
  Psi_eq = "No hi han posicions d'equilibri"
 
if V**2/(g*R_sc)<1:
  text2 = "${\Psi}_{equilibri}$ = ±"
else:
  text2 = "${\Psi}_{equilibri}$ = "
 
def equations(t,y):
    dy=np.zeros([2])
    dy[0] = y[1]
    dy[1] = (g/(l_o*(1-b*np.cos(y[0]))**(1./3)))*np.sin(y[0])-((V**2)/(R_sc*l_o*(1-b*np.cos(y[0]))**(1./3)))*np.sign(y[0]) - (2 * b * (y[1])**2 * np.sin(y[0]))/(3-3 * b * np.cos(y[0]))
 
    return dy
 
final_time = int(input("Temps final o durada del trajecte (s): "))
interval=30*final_time
time = np.linspace(0, final_time, interval)
 
if V**2/(g*R_sc)<1:
  Psi_equilibri_arr = round(0.98*Psi_eq,3)
  print("Angle d'equilibri per referència: ±"+str(Psi_eq)+"º",  colored(("[NO POT PASSAR DE ± 0.99 · \u03A8  ≈ ± "+str(Psi_equilibri_arr)+"... º]"),"red"))
else:
  print("No hi han posicions d'equilibri, és a dir, el pèndol/l'esquiador oscil·larà constantment a qualsevol angle sempre i quan no sigui extrem (70º-90º, 90º seria inclinar-se paral·lelament amb el terra | A partir de 70º l'esquiador pateix fins a 3 g s de força en cas el cas que la pista sigui horitzontal, això requereix molta força muscular.)")
 
initial_Psi = float(input("Angle inicial del pèndol/esquiador respecte de la vertical, \u03A6 (º): "))
P = float(input("Percentatge de flexió i extensió de les cames (0-100%): "))
b = P/100
y0 = [np.radians(initial_Psi), np.radians(0.0)]
 

res = solve_ivp(equations, (0,final_time), y0, method='RK45', t_eval=time, dense_output=True, rtol=1e-8, atol=1e-10)
 
t, psi, omega = res.t, res.y[0], res.y[1]

dense1 = res.sol
def dgamma(t, gamma):
    Psi, Omega = dense1(t)

    return [(V*np.sign(Psi))/(R_sc*np.cos(Psi))]

initial_gamma = [np.radians(0.0)]

gamma_res = solve_ivp(dgamma, (0,final_time), initial_gamma, method="RK45", t_eval = time, dense_output=True, rtol=1e-8, atol=1e-10)
gamma_times, gammas = gamma_res.t, gamma_res.y

dense2 = gamma_res.sol
def dy(t, y):
    Gamma = dense2(t)

    return [V*np.sin(Gamma)]

initial_y = [0.0]

y_res = solve_ivp(dy, (0,final_time), initial_y, method="RK45", t_eval = time, dense_output=True, rtol=1e-8, atol=1e-10)
y_times, y_skis = y_res.t, y_res.y[0]

pos_y= l_o*np.sin(psi)*(1-b*np.cos(psi))**(1./3) + y_skis
pos_z= l_o*np.cos(psi)*(1-b*np.cos(psi))**(1./3)
 
fig = plt.figure()
ax = plt.axes()
gca = plt.gca()
gca.set_aspect("equal")
plt.vlines(x= (max(y_skis))/2,ymin=0, ymax=2,colors="firebrick",linestyles="--",linewidth=1, zorder=0)
plt.hlines(y=0,xmin=-1, xmax=max(y_skis)+1,colors="black",linewidth=1,zorder=0)
plt.scatter(100,100,26,c="red", label="CM",zorder=2)
plt.xlabel("X [m]", fontsize=9, fontweight = "bold")
plt.ylabel("Z [m]", fontsize=9, fontweight = "bold")
plt.title('Moviment del pèndol centrífug invertit ')
gca.axes.xaxis.set_ticks([])
gca.axes.yaxis.set_ticks([l_o*(1-b)**(1./3)])
plt.ylim(-0.25,l_o)
plt.xlim(-0.25,max(y_skis)+0.25)
plt.legend(loc = "upper right", fontsize=8)
trans = plt.axes().transAxes
time_text1 = ax.text(0.01, 0.93, '', transform=trans, fontsize=7)
psi_text = ax.text(0.02, 0.85, '', transform=trans, fontsize=7)
length_text = ax.text(0.02, 0.77, '', transform=trans, fontsize=7)
line, = plt.plot([], [], 'k', lw=0.4, zorder=1)
skis_frontal, = plt.plot([], [], 'k', lw=2, zorder=1)
point, = plt.plot([],[], marker="o", c="red")
plt.close()

skis_Y_ini = -0.07*np.cos(psi) + y_skis
skis_Y_fin = 0.07*np.cos(psi) + y_skis
skis_Z_ini = 0.07*np.sin(psi)
skis_Z_fin = -0.07*np.sin(psi)
l = l_o * (1-b*np.cos(psi))**(1./3)

def init():
  line.set_data([], [])
  skis_frontal.set_data([], [])
  point.set_data([],[])
  time_text1.set_text('')
  psi_text.set_text('')
  length_text.set_text('')
  return line, skis_frontal, point, time_text1, psi_text, length_text

def animate(i):
  line.set_data([y_skis[i],pos_y[i]], [0,pos_z[i]])
  skis_frontal.set_data([skis_Y_ini[i], skis_Y_fin[i]], [skis_Z_ini[i], skis_Z_fin[i]])
  point.set_data([pos_y[i]],[pos_z[i]])
  time_text1.set_text('Temps = %.1f s' %(i/30))
  psi_text.set_text('\u03A6 = \u03A8 = %.2f º' %np.degrees(psi[i]))
  length_text.set_text('l = %.3f m' %l[i])
  return line, skis_frontal, point, time_text1, psi_text, length_text
 
anim = FuncAnimation(fig, animate,init_func=init, frames=interval, interval=33.3333334, blit=True)
 
if V**2/(g*R_sc)<1 and (initial_Psi > Psi_eq or initial_Psi < -Psi_eq):
  adv1 = "(L'esquiador ha perdut l'equilibri i ha caigut)"
else:
  adv1 = " "
 
print(colored(adv1,"red",attrs=['bold']))
HTML(anim.to_html5_video())

ADVERTÈNCIA: EL SIMULADOR ÉS VÀLID FINS APROXIMADAMENT 65º-70º
Alçada de l'esquiador (m): 1.77
Radi dels esquís (m): 14
Velocitat de l'esquiador durant el trajecte (m/s): 15
Temps final o durada del trajecte (s): 10
No hi han posicions d'equilibri, és a dir, el pèndol/l'esquiador oscil·larà constantment a qualsevol angle sempre i quan no sigui extrem (70º-90º, 90º seria inclinar-se paral·lelament amb el terra | A partir de 70º l'esquiador pateix fins a 3 g s de força en cas el cas que la pista sigui horitzontal, això requereix molta força muscular.)
Angle inicial del pèndol/esquiador respecte de la vertical, Φ (º): 65
Percentatge de flexió i extensió de les cames (0-100%): 75


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:96: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


In [4]:
import numpy as np
from numpy import sin,cos,sign,sqrt,arcsin,pi, arctan, tan
from scipy.integrate import solve_ivp
from matplotlib import pyplot as plt
from termcolor import colored
from matplotlib.pyplot import figure
import sympy as smp
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib import animation, rc
from IPython.display import HTML
 
print(colored("ADVERTÈNCIA: EL SIMULADOR ÉS VÀLID FINS APROXIMADAMENT 65º-70º","red"))
g = 9.807
height = float(input("Alçada de l'esquiador (m): "))
l = 0.56*height
R_sc = float(input("Radi dels esquís (m): "))
V = float(input("Velocitat de l'esquiador durant el trajecte (m/s): "))
 
if V**2/(g*R_sc)<1:
  Psi_eq = round(np.arcsin(V**2/(g*R_sc))*(180/np.pi), 3)
else:
  Psi_eq = "No hi han posicions d'equilibri"
 
if V**2/(g*R_sc)<1:
  text2 = "${\Psi}_{equilibri}$ = ±"
else:
  text2 = "${\Psi}_{equilibri}$ = "
 
def equations(t,y):
    dy=np.zeros([2])
    dy[0] = y[1]
    dy[1] = (g/l*A)*np.sin(A*y[0])-(V**2 * np.cos(A*y[0]) * np.sign(y[0]))/(l*R_sc*A*np.cos(y[0]))
 
    return dy
 
final_time = int(input("Temps final o durada del trajecte (s): "))
interval=30*final_time
time = np.linspace(0, final_time, interval)
 
if V**2/(g*R_sc)<1:
  Psi_equilibri_arr = round(0.98*Psi_eq,3)
  print("Angle d'equilibri per referència: ±"+str(Psi_eq)+"º",  colored(("[NO POT PASSAR DE ± 0.99 · \u03A8  ≈ ± "+str(Psi_equilibri_arr)+"... º]"),"red"))
else:
  print("No hi han posicions d'equilibri, és a dir, el pèndol/l'esquiador oscil·larà constantment a qualsevol angle sempre i quan no sigui extrem (70º-90º, 90º seria inclinar-se paral·lelament amb el terra | A partir de 70º l'esquiador pateix fins a 3 g s de força en cas el cas que la pista sigui horitzontal, això requereix molta força muscular.)")
 
initial_Psi = float(input("Angle inicial del pèndol/esquiador respecte de la vertical, \u03A6 (º): "))
print("Mínim percentatge d'angulació (l'esquí assoleix 90º): "+str((initial_Psi*100)/90)+" % (es recomana que sigui una mica més alt encara per evitar errors)")
print("Màxim percentatge d'angulació (no hi ha angulació): 100 %")
P = float(input("Percentatge d'angulació: "))
A = P/100
y0 = [np.radians(initial_Psi/A), np.radians(0.0)]
 
res = solve_ivp(equations, (0,final_time), y0, method='RK45', t_eval=time, dense_output=True, rtol=1e-8, atol=1e-10)
 
t, psi, omega = res.t, res.y[0], res.y[1]

dense1 = res.sol
def dgamma(t, gamma):
    Psi, Omega = dense1(t)

    return [(V*np.sign(Psi))/(R_sc*np.cos(Psi))]

initial_gamma = [np.radians(0.0)]

gamma_res = solve_ivp(dgamma, (0,final_time), initial_gamma, method="RK45", t_eval = time, dense_output=True, rtol=1e-8, atol=1e-10)
gamma_times, gammas = gamma_res.t, gamma_res.y

dense2 = gamma_res.sol
def dy(t, y):
    Gamma = dense2(t)

    return [V*np.sin(Gamma)]

initial_y = [0.0]

y_res = solve_ivp(dy, (0,final_time), initial_y, method="RK45", t_eval = time, dense_output=True, rtol=1e-8, atol=1e-10)
y_times, y_skis = y_res.t, y_res.y[0]

pos_x= l*np.sin(A*psi) + y_skis
pos_z= l*np.cos(A*psi)
 
fig = plt.figure()
ax = plt.axes()
gca = plt.gca()
gca.set_aspect("equal")
plt.vlines(x= (max(y_skis))/2,ymin=0, ymax=2,colors="firebrick",linestyles="--",linewidth=1, zorder=0)
plt.hlines(y=0,xmin=-1, xmax=max(y_skis)+1,colors="black",linewidth=1,zorder=0)
plt.scatter(100,100,26,c="red", label="CM",zorder=2)
plt.xlabel("X [m]", fontsize=9, fontweight = "bold")
plt.ylabel("Z [m]", fontsize=9, fontweight = "bold")
plt.title('Moviment del pèndol centrífug invertit ')
gca.axes.xaxis.set_ticks([])
gca.axes.yaxis.set_ticks([l])
plt.ylim(-0.25,l+0.2)
plt.xlim(-0.25,max(y_skis)+0.25)
plt.legend(loc = "upper right", fontsize=8)
trans = plt.axes().transAxes
time_text1 = ax.text(0.01, 0.93, '', transform=trans, fontsize=7)
psi_text = ax.text(0.02, 0.77, '', transform=trans, fontsize=7)
phi_text = ax.text(0.02, 0.85, '', transform=trans, fontsize=7)
line, = plt.plot([], [], 'k', lw=0.4, zorder=1)
skis_frontal, = plt.plot([], [], 'k', lw=2, zorder=1)
point, = plt.plot([],[], marker="o", c="red")
plt.close()

skis_Y_ini = -0.07*np.cos(psi) + y_skis
skis_Y_fin = 0.07*np.cos(psi) + y_skis
skis_Z_ini = 0.07*np.sin(psi)
skis_Z_fin = -0.07*np.sin(psi)

def init():
  line.set_data([], [])
  skis_frontal.set_data([], [])
  point.set_data([],[])
  time_text1.set_text('')
  psi_text.set_text('')
  phi_text.set_text('')
  return line, skis_frontal, point, time_text1, psi_text, phi_text

def animate(i):
  line.set_data([y_skis[i],pos_x[i]], [0,pos_z[i]])
  skis_frontal.set_data([skis_Y_ini[i], skis_Y_fin[i]], [skis_Z_ini[i], skis_Z_fin[i]])
  point.set_data([pos_x[i]],[pos_z[i]])
  time_text1.set_text('Temps = %.1f s' %(i/30))
  psi_text.set_text('\u03A8 = %.2f º' %np.degrees(psi[i]))
  phi_text.set_text('\u03A6 = %.2f º' %np.degrees(A*psi[i]))
  return line, skis_frontal, point, time_text1, psi_text, phi_text
 
anim = FuncAnimation(fig, animate,init_func=init, frames=interval, interval=33.3333334, blit=True)
 
if V**2/(g*R_sc)<1 and (initial_Psi > Psi_eq or initial_Psi < -Psi_eq):
  adv1 = "(L'esquiador ha perdut l'equilibri i ha caigut)"
else:
  adv1 = " "
 
print(colored(adv1,"red",attrs=['bold']))
HTML(anim.to_html5_video())

ADVERTÈNCIA: EL SIMULADOR ÉS VÀLID FINS APROXIMADAMENT 65º-70º
Alçada de l'esquiador (m): 1.77


KeyboardInterrupt: ignored

In [5]:
import numpy as np
from numpy import sin,cos,sign,sqrt,arcsin,pi, arctan, tan
from scipy.integrate import solve_ivp
from matplotlib import pyplot as plt
from termcolor import colored
from matplotlib.pyplot import figure
import sympy as smp
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib import animation, rc
from IPython.display import HTML

print(colored("ADVERTÈNCIA: EL SIMULADOR ÉS VÀLID FINS APROXIMADAMENT 65º-70º","red"))
g = 9.807
height = float(input("Alçada de l'esquiador (m): "))
l_o = 0.56*height
R_sc = float(input("Radi dels esquís (m): "))
V = float(input("Velocitat de l'esquiador durant el trajecte (m/s): "))
 
if V**2/(g*R_sc)<1:
  Psi_eq = round(np.arcsin(V**2/(g*R_sc))*(180/np.pi), 3)
else:
  Psi_eq = "No hi han posicions d'equilibri"
 
if V**2/(g*R_sc)<1:
  text2 = "${\Psi}_{equilibri}$ = ±"
else:
  text2 = "${\Psi}_{equilibri}$ = "
 
def equations(t,y):
    dy=np.zeros([2])
    dy[0] = y[1]
    dy[1] = (g/(A * l_o * (1-b * np.cos(A*y[0]))**(1./3)))*np.sin(A*y[0])-(V**2 * np.cos(A*y[0]) * np.sign(y[0]))/(l_o * (1-b * np.cos(A*y[0]))**(1./3) * R_sc * A * np.cos(y[0])) - (2 * b * (y[1])**2 * np.sin(A*y[0]))/(3*A-3* A * b * np.cos(A*y[0]))
 
    return dy
 
final_time = int(input("Temps final o durada del trajecte (s): "))
interval=30*final_time
time = np.linspace(0, final_time, interval)
 
if V**2/(g*R_sc)<1:
  Psi_equilibri_arr = round(0.98*Psi_eq,3)
  print("Angle d'equilibri per referència: ±"+str(Psi_eq)+"º",  colored(("[NO POT PASSAR DE ± 0.99 · \u03A8  ≈ ± "+str(Psi_equilibri_arr)+"... º]"),"red"))
else:
  print("No hi han posicions d'equilibri, és a dir, el pèndol/l'esquiador oscil·larà constantment a qualsevol angle sempre i quan no sigui extrem (70º-90º, 90º seria inclinar-se paral·lelament amb el terra | A partir de 70º l'esquiador pateix fins a 3 g s de força en cas el cas que la pista sigui horitzontal, això requereix molta força muscular.)")
 
initial_Psi = float(input("Angle inicial del pèndol/esquiador respecte de la vertical, \u03A6 (º): "))
P_1 = float(input("Percentatge de flexió i extensió de les cames (0-100%): "))
b = P_1/100
print("Mínim percentatge d'angulació (l'esquí assoleix 90º): "+str((initial_Psi*100)/90)+" % (es recomana que sigui una mica més alt encara per evitar errors)")
print("Màxim percentatge d'angulació (no hi ha angulació): 100 %")
P_2 = float(input("Percentatge d'angulació: "))
A = P_2/100
y0 = [np.radians(initial_Psi/A), np.radians(0.0)]
 

res = solve_ivp(equations, (0,final_time), y0, method='RK45', t_eval=time, dense_output=True, rtol=1e-8, atol=1e-10)
 
t, psi, omega = res.t, res.y[0], res.y[1]

dense1 = res.sol
def dgamma(t, gamma):
    Psi, Omega = dense1(t)

    return [(V*np.sign(Psi))/(R_sc*np.cos(Psi))]

initial_gamma = [np.radians(0.0)]

gamma_res = solve_ivp(dgamma, (0,final_time), initial_gamma, method="RK45", t_eval = time, dense_output=True, rtol=1e-8, atol=1e-10)
gamma_times, gammas = gamma_res.t, gamma_res.y

dense2 = gamma_res.sol
def dx(t, x):
    Gamma = dense2(t)

    return [V*np.cos(Gamma)]

initial_x = [0.0]

x_res = solve_ivp(dx, (0,final_time), initial_x, method="RK45", t_eval = time, dense_output=True, rtol=1e-8, atol=1e-10)
x_times, x_skis = x_res.t, x_res.y

def dy(t, y):
    Gamma = dense2(t)

    return [V*np.sin(Gamma)]

initial_y = [0.0]

y_res = solve_ivp(dy, (0,final_time), initial_y, method="RK45", t_eval = time, dense_output=True, rtol=1e-8, atol=1e-10)
y_times, y_skis = y_res.t, y_res.y[0]

fig = plt.figure()
ax = plt.axes()
gca = plt.gca()
gca.set_aspect("equal")
plt.vlines(x= (max(y_skis))/2,ymin=0, ymax=2,colors="firebrick",linestyles="--",linewidth=1, zorder=0)
plt.hlines(y=0,xmin=-1, xmax=max(y_skis)+1,colors="black",linewidth=1,zorder=0)
plt.scatter(100,100,26,c="red", label="CM",zorder=2)
plt.xlabel("X [m]", fontsize=9, fontweight = "bold")
plt.ylabel("Z [m]", fontsize=9, fontweight = "bold")
plt.title('Moviment del pèndol centrífug invertit ')
gca.axes.xaxis.set_ticks([])
gca.axes.yaxis.set_ticks([l_o*(1-b)**(1./3)])
plt.ylim(-0.25,l_o)
plt.xlim(-0.25,max(y_skis)+0.25)
plt.legend(loc = "upper right", fontsize=8)
trans = plt.axes().transAxes
time_text1 = ax.text(0.01, 0.93, '', transform=trans, fontsize=7)
psi_text = ax.text(0.02, 0.85, '', transform=trans, fontsize=7)
phi_text = ax.text(0.02, 0.77, '', transform=trans, fontsize=7)
length_text = ax.text(0.02, 0.69, '', transform=trans, fontsize=7)
line, = plt.plot([], [], 'k', lw=0.4, zorder=1)
skis_frontal, = plt.plot([], [], 'k', lw=2, zorder=1)
point, = plt.plot([],[], marker="o", c="red")
plt.close()

skis_Y_ini = -0.07*np.cos(psi) + y_skis
skis_Y_fin = 0.07*np.cos(psi) + y_skis
skis_Z_ini = 0.07*np.sin(psi)
skis_Z_fin = -0.07*np.sin(psi)
l = l_o * (1-b*np.cos(A*psi))**(1./3)
pos_y= l*np.sin(A*psi) + y_skis
pos_z= l*np.cos(A*psi)

def init():
  line.set_data([], [])
  skis_frontal.set_data([], [])
  point.set_data([],[])
  time_text1.set_text('')
  psi_text.set_text('')
  phi_text.set_text('')
  length_text.set_text('')
  return line, skis_frontal, point, time_text1, psi_text, phi_text, length_text

def animate(i):
  line.set_data([y_skis[i],pos_y[i]], [0,pos_z[i]])
  skis_frontal.set_data([skis_Y_ini[i], skis_Y_fin[i]], [skis_Z_ini[i], skis_Z_fin[i]])
  point.set_data([pos_y[i]],[pos_z[i]])
  time_text1.set_text('Temps = %.1f s' %(i/30))
  psi_text.set_text('\u03A8 = %.2f º' %np.degrees(psi[i]))
  phi_text.set_text('\u03A6 = %.2f º' %np.degrees(A*psi[i]))
  length_text.set_text('l = %.3f m' %l[i])
  return line, skis_frontal, point, time_text1, psi_text, phi_text, length_text
 
anim = FuncAnimation(fig, animate,init_func=init, frames=interval, interval=33.3333334, blit=True)
 
if V**2/(g*R_sc)<1 and (initial_Psi > Psi_eq or initial_Psi < -Psi_eq):
  adv1 = "(L'esquiador ha perdut l'equilibri i ha caigut)"
else:
  adv1 = " "
 
print(colored(adv1,"red",attrs=['bold']))
HTML(anim.to_html5_video())

ADVERTÈNCIA: EL SIMULADOR ÉS VÀLID FINS APROXIMADAMENT 65º-70º
Alçada de l'esquiador (m): 1.77
Radi dels esquís (m): 14
Velocitat de l'esquiador durant el trajecte (m/s): 14
Temps final o durada del trajecte (s): 10
No hi han posicions d'equilibri, és a dir, el pèndol/l'esquiador oscil·larà constantment a qualsevol angle sempre i quan no sigui extrem (70º-90º, 90º seria inclinar-se paral·lelament amb el terra | A partir de 70º l'esquiador pateix fins a 3 g s de força en cas el cas que la pista sigui horitzontal, això requereix molta força muscular.)
Angle inicial del pèndol/esquiador respecte de la vertical, Φ (º): 65
Percentatge de flexió i extensió de les cames (0-100%): 75
Mínim percentatge d'angulació (l'esquí assoleix 90º): 72.22222222222223 % (es recomana que sigui una mica més alt encara per evitar errors)
Màxim percentatge d'angulació (no hi ha angulació): 100 %
Percentatge d'angulació: 90


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
